In [1]:
import os
import cv2
import numpy as np
from datetime import datetime

# Parámetros
future_steps = 4  # Número de imágenes futuras a predecir
sequence_length = 8  # Número de imágenes en la secuencia de entrada
height, width, channels = 480, 480, 3  # Tamaño de las imágenes
image_folder = '../data/Images_test/'  # Cambia esto a la ruta de tu carpeta
forecast_folder = '../data/Images_Forecast'  # Carpeta donde se guardarán predicciones

# Crear la carpeta de predicción si no existe
os.makedirs(forecast_folder, exist_ok=True)

def get_timestamp_from_filename(filename):
    """Extrae el timestamp del nombre del archivo."""
    base = os.path.basename(filename)
    try:
        timestamp_str = base.split('_')[3][1:].split('.')[0]  # Elimina el sufijo '.jpg'
        return datetime.strptime(timestamp_str, '%Y%m%d%H%M%S')
    except Exception as e:
        raise ValueError(f"No se pudo extraer el timestamp de: {filename}. Error: {e}")

def load_and_preprocess_image(image_path, target_size=(height, width)):
    """Carga y preprocesa una imagen."""
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"No se puede leer la imagen: {image_path}")
    image = cv2.resize(image, target_size)
    image = image.astype('float32') / 255.0  # Normalización RGB
    return image

def load_image_sequences(folder, sequence_length, future_steps):
    """Carga secuencias de imágenes para entrenamiento/predicción."""
    image_files = sorted(
        [os.path.join(folder, i) for i in os.listdir(folder) if i.endswith('.jpg')],
        key=get_timestamp_from_filename,
    )
    X, y = [], []
    timestamps = [get_timestamp_from_filename(f) for f in image_files]

    # Verificar intervalos de tiempo de 10 minutos
    valid_indices = []
    for i in range(len(timestamps) - sequence_length - future_steps + 1):
        is_valid = all(
            (timestamps[i + j + 1] - timestamps[i + j]).total_seconds() == 600
            for j in range(sequence_length + future_steps - 1)
        )
        if is_valid:
            valid_indices.append(i)

    # Cargar las secuencias
    for idx in valid_indices:
        sequence_images = [
            load_and_preprocess_image(image_files[idx + j]) for j in range(sequence_length)
        ]
        future_images = [
            load_and_preprocess_image(image_files[idx + sequence_length + k])
            for k in range(future_steps)
        ]
        X.append(sequence_images)
        y.append(future_images)

    return np.array(X), np.array(y)

# Cargar secuencias de imágenes
try:
    X, y = load_image_sequences(image_folder, sequence_length, future_steps)

    # Mostrar información de las secuencias cargadas
    print(f"Secuencias cargadas: {X.shape}")
    print(f"Imágenes futuras: {y.shape}")

    # Guardar las secuencias generadas como ejemplo
    np.save(os.path.join(forecast_folder, "X_sequences.npy"), X)
    np.save(os.path.join(forecast_folder, "y_sequences.npy"), y)
except ValueError as e:
    print(f"Error: {e}")


Secuencias cargadas: (103, 8, 480, 480, 3)
Imágenes futuras: (103, 4, 480, 480, 3)


In [8]:
import os
import cv2
import numpy as np
from datetime import datetime, timedelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import preprocess_input

# Configuración de rutas
input_folder = '../data/Images/'  # Carpeta con las imágenes originales
output_folder = '../data/Images_Forecast/'  # Carpeta para las imágenes predichas
os.makedirs(output_folder, exist_ok=True)

# Dimensiones de las imágenes
height, width, channels = 449, 593, 3
sequence_length = 10  # Número de imágenes (1 día de datos)

# Cargar el extractor de características VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(height, width, channels))
flatten_layer = Flatten()(base_model.output)
feature_extractor = Model(inputs=base_model.input, outputs=flatten_layer)

# Función para cargar y normalizar una imagen
def load_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (width, height))  # Ajustar tamaño al esperado
    img = preprocess_input(img)  # Preprocesar para VGG16
    return img

# Función para extraer características de secuencias de imágenes
def extract_features(image_sequences):
    num_sequences = len(image_sequences)
    features = np.zeros((num_sequences, sequence_length, feature_extractor.output_shape[1]))

    for i in range(num_sequences):
        for j in range(sequence_length):
            img = image_sequences[i, j]
            img = np.expand_dims(img, axis=0)  # Expandir dimensión del lote
            features[i, j] = feature_extractor.predict(img).squeeze()
    return features

# Preparar datos para entrenamiento
def prepare_data(image_paths, sequence_length, future_steps):
    images = [load_image(path) for path in image_paths]
    X, y = [], []
    for i in range(len(images) - sequence_length - future_steps + 1):
        X.append(images[i:i + sequence_length])  # Secuencia de entrada
        y.append(images[i + sequence_length:i + sequence_length + future_steps])  # Secuencia de salida
    return np.array(X), np.array(y)

# Modelo LSTM dinámico
def build_lstm_model(sequence_length, future_steps):
    model = Sequential()
    model.add(LSTM(512, input_shape=(sequence_length, feature_extractor.output_shape[1]), return_sequences=True))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(future_steps * height * width * channels, activation='linear'))
    model.add(Reshape((future_steps, height, width, channels)))  # Salida con múltiples imágenes
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Leer y ordenar los nombres de imágenes por timestamp
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
timestamps = {datetime.strptime(f.split('_s')[1].split('.jpg')[0], '%Y%m%d%H%M%S'): f for f in image_files}
timestamps_sorted = sorted(timestamps.keys())

# Mostrar rango de fechas disponibles
print("Rango disponible de imágenes:")
print(f"Inicia: {timestamps_sorted[0]}")
print(f"Termina: {timestamps_sorted[-1]}")

# Generar timestamps esperados en intervalos de 10 minutos
expected_timestamps = []
current_time = timestamps_sorted[0]
while current_time <= timestamps_sorted[-1]:
    expected_timestamps.append(current_time)
    current_time += timedelta(minutes=10)

# Identificar timestamps faltantes
missing_timestamps = [ts for ts in expected_timestamps if ts not in timestamps]

# Agrupar timestamps faltantes en rangos consecutivos
def group_missing_timestamps(missing_timestamps):
    if not missing_timestamps:
        return []

    ranges = []
    start = missing_timestamps[0]
    prev = missing_timestamps[0]

    for ts in missing_timestamps[1:]:
        if ts - prev > timedelta(minutes=10):
            ranges.append((start, prev))
            start = ts
        prev = ts
    ranges.append((start, prev))

    return ranges

missing_ranges = group_missing_timestamps(missing_timestamps)

# Mostrar al usuario los rangos faltantes
if missing_ranges:
    print("\nTimestamps faltantes agrupados en rangos:")
    for i, (start, end) in enumerate(missing_ranges, start=1):
        print(f"{i}: Desde: {start} Hasta: {end}")

    # Permitir que el usuario seleccione un rango por índice
    try:
        selected_index = int(input("\nSeleccione el índice del rango a completar (por número): "))
        if selected_index < 1 or selected_index > len(missing_ranges):
            raise ValueError("Índice fuera de rango.")

        # Obtener el rango seleccionado
        start_date, end_date = missing_ranges[selected_index - 1]
        print(f"\nRango seleccionado: Desde {start_date} Hasta {end_date}")

        # Filtrar los timestamps que caen dentro del rango seleccionado
        timestamps_to_predict = [ts for ts in missing_timestamps if start_date <= ts <= end_date]
        future_steps = len(timestamps_to_predict)  # Ajustar dinámicamente los futuros pasos
        print(f"\nSe predecirán {future_steps} imágenes en el rango seleccionado.")
    except ValueError as e:
        print(f"Error: {e}. Por favor, ingrese un número válido.")
else:
    print("No hay vacíos que completar en este momento.")
    timestamps_to_predict = []

# Entrenar y predecir dinámicamente para llenar el vacío
def train_and_predict_dynamic(images, timestamps_to_predict, feature_extractor, future_steps):
    # Determinar cuántas imágenes están disponibles
    available_images = len(images)
    if available_images < sequence_length:
        print(f"No hay suficientes imágenes para entrenar. Imágenes disponibles: {available_images}")
        return

    # Ajustar el número de imágenes de entrada al mínimo disponible
    adjusted_sequence_length = min(sequence_length, available_images)
    adjusted_images = images[-adjusted_sequence_length:]  # Últimas imágenes disponibles
    print(f"Usando {len(adjusted_images)} imágenes para el entrenamiento.")

    # Preparar datos para entrenamiento
    image_paths = [os.path.join(input_folder, timestamps[ts]) for ts in timestamps_sorted if ts in timestamps]
    X, y = prepare_data(image_paths[-adjusted_sequence_length:], adjusted_sequence_length, future_steps)

    if len(X) == 0 or len(y) == 0:
        print(f"No se generaron suficientes secuencias de datos para entrenar el modelo.")
        return

    X_features = extract_features(X)

    # Construir el modelo
    model = build_lstm_model(adjusted_sequence_length, future_steps)

    # Entrenar el modelo
    reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5, min_lr=1e-6)
    model.fit(X_features, y, epochs=20, batch_size=16, callbacks=[reduce_lr])

    # Realizar predicciones
    predict_image_sequence(model, images, timestamps_to_predict, feature_extractor, future_steps)

# Función para predecir una secuencia de imágenes
def predict_image_sequence(model, images, timestamps_to_predict, feature_extractor, future_steps):
    for i in range(0, len(timestamps_to_predict), future_steps):
        # Usar las últimas imágenes disponibles
        recent_images = images[-sequence_length:] if len(images) >= sequence_length else images
        recent_images = np.array(recent_images)

        # Extraer características de las últimas imágenes
        recent_features = extract_features(recent_images[np.newaxis, :, :, :, :], feature_extractor)

        # Predecir la secuencia de imágenes
        predicted_sequence = model.predict(recent_features)[0]  # Secuencia de imágenes predichas

        for step, predicted_image in enumerate(predicted_sequence):
            if i + step >= len(timestamps_to_predict):  # Evitar índices fuera de rango
                break
            timestamp = timestamps_to_predict[i + step]
            predicted_image = np.clip(predicted_image * 255, 0, 255).astype(np.uint8)  # Desnormalizar

            # Guardar cada imagen predicha
            predicted_file_name = f"OR_ABI-L2-ACMF-M6_G16_s{timestamp.strftime('%Y%m%d%H%M%S')}.jpg"
            cv2.imwrite(os.path.join(output_folder, predicted_file_name), predicted_image)

            # Añadir la imagen predicha a la lista de imágenes
            images.append(predicted_image)

    print(f"\nSe predijeron {len(timestamps_to_predict)} imágenes en el rango seleccionado.")
    print(f"Imágenes guardadas en: {output_folder}")

# Realizar predicciones si hay vacíos seleccionados
if timestamps_to_predict:
    train_and_predict_dynamic(image_files, timestamps_to_predict, feature_extractor, future_steps)
else:
    print("No hay vacíos que completar en este momento.")


Rango disponible de imágenes:
Inicia: 2023-07-01 05:00:21
Termina: 2023-07-10 18:20:20

Timestamps faltantes agrupados en rangos:
1: Desde: 2023-07-06 05:00:21 Hasta: 2023-07-06 16:50:21
2: Desde: 2023-07-07 05:00:21 Hasta: 2023-07-07 16:50:21
3: Desde: 2023-07-07 18:30:21 Hasta: 2023-07-10 18:10:21

Rango seleccionado: Desde 2023-07-06 05:00:21 Hasta 2023-07-06 16:50:21

Se predecirán 72 imágenes en el rango seleccionado.
Usando 10 imágenes para el entrenamiento.
No se generaron suficientes secuencias de datos para entrenar el modelo.
